# Webinar n°2: Pyleecan Advanced - Part 3: Mesh and Solution

This notebook is the support of the second out of three webinars organized by the association [Green Forge Coop](https://www.linkedin.com/company/greenforgecoop/about/) and the UNICAS University. 

The webinars schedule is:
- Friday 16th October 15h-17h (GMT+2): How to use pyleecan (basics)? Pyleecan basics, call of FEMM, use of the GUI
- Friday 30th October 15h-17h (GMT+1): How to use pyleecan (advanced)? Optimization tools, meshing, plot commands
- Friday 6th November 15h-17h (GMT+1): How to contribute to pyleecan? Github projects, Object Oriented Programming

Speakers: Pierre Bonneel, Hélène Toubin, Raphaël Pile from EOMYS.

This webinar will be recorded and the video will be shared on [pyleecan.org](https://pyleecan.org/tutorials.html)

To use this notebook please:
- Install Anaconda
- In Anaconda Prompt run the command "pip install pyleecan"
- Install the latest version of [femm](http://www.femm.info/wiki/Download) (windows only)
- In Anaconda Navigator, lanch Jupyter Notebook
- Jupyter Notebook should open a tab in your web brower, select this notebook to open it

For this part of the webinar you will also need to install gmsh (pip install gmsh)

To check if everything is correctly set, please run the following cell (WARNING: the webinar use pyleecan 1.0.2 and SciDataTool 1.1.5):

In [ ]:
%matplotlib notebook

# Print version of all packages
import pyleecan
print("Pyleecan version: "+pyleecan.__version__)

# Load the machine
from os.path import join
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR

IPMSM_A = load(join(DATA_DIR, "Machine", "IPMSM_A.json"))
IPMSM_A.plot()

# Check FEMM installation
from pyleecan.Classes._FEMMHandler import FEMMHandler

femm = FEMMHandler()
femm.openfemm(0)

import gmsh

# 1) Introduction
This tutorial shows the different possibilities allowed by the MeshSolution module. The main purpose of the module is to store Finite Element (FE) results in a way adapted to Pyleecan architecture. Today, it is mainly dedicated to store 2D electromagnetic solution computed with FEMM, but the goal is to generalize to any physics that could be included inside Pyleecan.
 
The notebook related to this tutorial is available on [GitHub](https://github.com/Eomys/pyleecan/tree/master/Tutorials/tuto_MeshSolution.ipynb).

This tutorial is for people who wish to understand in depth how this module works, and potentially contribute to the development of the code.

The idea is to make a link between classic meshmakers (for example Gmsh) and the different physics in Pyleecan. Thus, a dedicated object is necessary. The goal is to to __simplify our lives__.  

# 2) Demo with FEMM results
The aim of this section is to show how MeshSolution objects are used in Pyleecan to post-process FE results. 

## Plots

In [ ]:
# Run the FEMM simulation from Webinar 1
from numpy import ones, pi, array, linspace, cos, sqrt
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.InputCurrent import InputCurrent
from pyleecan.Classes.MagFEMM import MagFEMM
from os.path import join
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR

# Create the Simulation
IPMSM_A = load(join(DATA_DIR, "Machine", "IPMSM_A.json"))
simu_femm = Simu1(name="Webinar_1_MagFemm", machine=IPMSM_A)   

# Definition of the magnetic simulation (FEMM with symmetry and sliding band)
simu_femm.mag = MagFEMM(
    type_BH_stator=0,
    type_BH_rotor=0,
    is_periodicity_a=True,
    is_periodicity_t=True,
    nb_worker=4,
    Kgeo_fineness=1,
)
# Run only Magnetic module
simu_femm.elec = None
simu_femm.force = None
simu_femm.struct = None
# Definition of the enforced output of the electrical module
simu_femm.input = InputCurrent(
    Na_tot=252 * 8,
    Nt_tot=64,
    N0=1000,
)
# Set Id/Iq according to I0/Phi0
simu_femm.input.set_Id_Iq(I0=250 / sqrt(2), Phi0=140*pi/180)

To enable the FE results saving, the option to activate is "is_get_mesh": 

In [ ]:
simu_femm.mag.is_get_mesh=True # TO save FEMM mesh and results into a MeshSolution object

out_femm = simu_femm.run()

Now, the magnetic FEA results can be plotted. First, the mesh from FEMM can be displayed:

In [ ]:
out_femm.mag.meshsolution.plot_mesh()

One or several sub-part(s) of the mesh can be extracted:

In [ ]:
out_femm.mag.meshsolution.plot_mesh(group_names="stator")

In [ ]:
out_femm.mag.meshsolution.plot_mesh(group_names=["airgap", "stator_windings"])

The intersection of the mesh can be extracted as well:

In [ ]:
out_femm.mag.meshsolution.plot_mesh(group_names=["stator", "/", "airgap", "stator_windings"])

In [ ]:
out_femm.mag.meshsolution.plot_mesh(group_names=["stator",  "airgap", "/", "stator_windings"])

Then, the electromagnetic solution can be displayed:

In [ ]:
out_femm.mag.meshsolution.plot_contour(label="B")

 Moreover, the solution can be extracted on a specific area.

In [ ]:
out_femm.mag.meshsolution.plot_contour(label="H", group_names="stator")

There is a method to plot solution as "arrows" (similar to the quiver method from matplotlib or matlab):

In [ ]:
out_femm.mag.meshsolution.plot_glyph(label="B", group_names="stator")

## Extract and post-process data

Several methods have been developed for the MeshSolution class in order to load the results regardless of the type of objects. For instance, I can extract the magnetic flux computed in every cells (triangles) of the magnetostatic mesh:

In [ ]:
B = out_femm.mag.meshsolution.get_field(label='B')
B.shape

A new MeshSolution object can be created from the group definition. For example, the following magnetic flux density size has been reduce:

In [ ]:
group_stator = out_femm.mag.meshsolution.get_group("stator")
B_s = group_stator.get_field(label='B')
H_s = group_stator.get_field(label='H')
B_s.shape

Then, operations can be performed on the solution of this group, and plotted. It is worth noting that several type of Solution objects can co-exist in the same MeshSolution object.

In [ ]:
import numpy as np
from pyleecan.Classes.SolutionMat import SolutionMat

w_mag = np.multiply(B_s,H_s)/2

w_axis = dict()
w_axis["time"] = B_s.shape[0]
w_axis["indice"] = B_s.shape[1]
w_axis["component"] = B_s.shape[2]

my_vec_solution = SolutionMat(
    label="w_mag",
    type_cell="triangle",
    field=w_mag,
    axis=w_axis,
)
group_stator.solution.append(my_vec_solution)
group_stator.plot_contour(label="w_mag")

Operations can also be performed on the mesh.

In [ ]:
nodes_s = group_stator.get_mesh().get_point()
nodes_s.shape

For example, a rotation the mesh:

In [ ]:
th = np.pi
R = np.array([[np.cos(th), -np.sin(th)], [np.sin(th), np.cos(th)]])
nodes_s = np.dot(nodes_s, R)
group_stator.mesh[0].point.coordinate = nodes_s
group_stator.plot_mesh()

Previous plot still work !

In [ ]:
group_stator.plot_contour(label="w_mag")

# 3) Advanced Mesh and Solution definition

## Defining yourself a Mesh object

At the date of this webinar, there are two different types of Mesh objects: MeshMat and MeshVTK. 
- MeshMat object is designed to ease postprocessing. It enables access to important values (connectivity, nodes) and to defined interpolation methods. (numpy.array)
- MeshVTK is designed to ease vizualization, by relying on existing librairy pyvista.

The bridge between the two type of class is available with dedicated convert methods. 

### Defining a MeshMat object

Although every features should be automatically initialized/defined in Pyleecan, we are going to define by hand some of the objects in order to introduce the basics principle of the MeshSolution module.

In [ ]:
from pyleecan.Classes.MeshMat import MeshMat
from pyleecan.Classes.PointMat import PointMat
from pyleecan.Classes.CellMat import CellMat
from pyleecan.Classes.MeshSolution import MeshSolution

mesh = MeshMat(dimension=3)
mesh.point = PointMat()
mesh.point.add_point([0, 0, 0])
mesh.point.add_point([0, 1, 0])
mesh.point.add_point([1, 0, 0])
mesh.point.add_point([1, 1, 0])
mesh.point.add_point([2, 1, 0])

mesh.cell["triangle"] = CellMat(nb_pt_per_cell=3)
mesh.add_cell([0, 1, 2], "triangle")
mesh.add_cell([1, 2, 3], "triangle")
mesh.add_cell([2, 3, 4], "triangle")

MSol = MeshSolution(mesh=[mesh])

MSol.plot_mesh()

## Defining a SolutionMat object and plot
The MeshSolution object allows to make the link between data (such as FE results) and the corresponding mesh stored in a Mesh object. Thus, all the plot and post-processing methods should be available in the MeshSolution class.

Today, the main post-processing are the plots (such as plot_contour and plot_glyph).

Here is an example with plot_contour: a scalar field is defined by giving its values all points of the mesh. 

In [ ]:
import numpy as np
from pyleecan.Classes.SolutionMat import SolutionMat

axis_dct = dict()
axis_dct["indice"] = 5
axis_dct["time"] = 1
field = np.array([[0,1,2,3,4]])

my_solution = SolutionMat(
    label="my_field",
    type_cell="point",
    field=field,
    indice=[0, 1, 2, 3, 4],
    axis=axis_dct,
)
MSol.solution.append(my_solution)
MSol.plot_contour()

The notion of axis allows to correctly extract values as it would be with SciDataTool objects -> same way to call methods in SolutionMat/SolutionData/SolutionVector. 

Using SolutionMat, one can also defined a vector field by using an additional axis "component".

In [ ]:
new_axis_dct = dict()

new_axis_dct["time"] = 10
new_axis_dct["indice"] = 5
new_axis_dct["component"] = 2
vector = np.ones((10,5,2))

my_vec_solution = SolutionMat(
    label="my_vector",
    type_cell="point",
    field=vector,
    indice=[0, 1, 2, 3, 4], # optional today, but field size must match with the number of point/cell.
    axis=new_axis_dct,
)
MSol.solution.append(my_vec_solution)
MSol.plot_glyph(label="my_vector", is_point_arrow=True, factor=1/10)

In this example, a 2D field is defined on a 3D mesh. The mesh and the field has distinct "dimension" attributes. It allows us to limit the memory space when possible. 

In order to have more details about the intialization of SolutionData/SolutionVector objects, see SciDataTool part and build_meshsolution() method from MagFEMM. 

## Import an external Mesh

First, we are going to generate a mesh with gmsh coupling.

In [ ]:
from pyleecan.Classes.LamSlotMulti import LamSlotMulti
from pyleecan.Classes.SlotW10 import SlotW10
from pyleecan.Classes.SlotW22 import SlotW22
from pyleecan.Classes.NotchEvenDist import NotchEvenDist
from pyleecan.Functions.GMSH.gen_lam_3D_mesh_GMSH import gen_lam_3D_mesh_GMSH

import matplotlib.pyplot as plt

from os import getcwd
from os.path import join
save_path = getcwd() # Get notebook directory

# Rotor definition
rotor = LamSlotMulti(
    Rint=0.2, Rext=0.7, is_internal=True, is_stator=False, L1=0.9, Nrvd=2, Wrvd=0.05
)

# Reference Slot
Zs = 8
Slot1 = SlotW10(
    Zs=Zs, W0=50e-3, H0=30e-3, W1=100e-3, H1=30e-3, H2=100e-3, W2=120e-3
)
Slot2 = SlotW22(Zs=Zs, W0=pi / 12, H0=50e-3, W2=pi / 6, H2=125e-3)

# Reference slot are duplicated to get 4 of each in alternance
slot_list = list()
for ii in range(Zs // 2):
    slot_list.append(SlotW10(init_dict=Slot1.as_dict()))
    slot_list.append(SlotW22(init_dict=Slot2.as_dict()))
rotor.slot_list = slot_list
# Set slot position as linspace
rotor.alpha = linspace(0, 2 * pi, 8, endpoint=False) + pi / Zs

# Set evenly distributed notches
slot3 = SlotW10(Zs=Zs // 2, W0=40e-3, W1=40e-3, W2=40e-3, H0=0, H1=0, H2=25e-3)
notch = NotchEvenDist(notch_shape=slot3, alpha=2 * pi / Zs)
rotor.notch = [notch]

# Plot the lamination
rotor.plot()

# Generate the gmsh equivalent
gen_lam_3D_mesh_GMSH(
    lamination=rotor,
    save_path=join(save_path, "fig_11_gmsh_SlotMulti.msh"),
    sym=4,
    mesh_size=20e-3,
    Nlayer=20,
    display=False,
)

Today, Pyleecan mainly relies on the meshio librairy to convert any type of mesh file into a .vtk which is readable by pyvista. Any contribution on this topic is welcome.

In [ ]:
#Convert to vtk with meshio
import meshio
m = meshio.Mesh.read(join(save_path, "fig_11_gmsh_SlotMulti.msh"))
m.write('my_mesh.vtk')

Finally, the .vtk file can be loaded with a MeshVTK object.

In [ ]:
# Import in Pyleecan with MeshVTK
from pyleecan.Classes.MeshVTK import MeshVTK
mesh_import = MeshVTK(
    path=save_path,
    name="my_mesh",
)
MSol_import = MeshSolution(mesh=[mesh_import])
MSol_import.plot_mesh()

Thanks for following this tutorial ! :-)